In [11]:
import wave as wave
import numpy as np
import pyroomacoustics as pa
import scipy.signal as sp

def calculate_steering_vector(mic_alignments, source_locations, freqs, sound_speed=340, is_use_far=True):
    n_channels = np.shape(mic_alignments)

    n_sources = np.shape(source_locations)

    if is_use_far == True:
        norm_source_locations = source_locations/np.linalg.norm(source_locations, 2, axis=0, keepdims=True)
        # numpyの機能で自動で計算する次元を合わせてくれる？らしい
        # アインシュタインの縮約記法を理解する必要あり
        # あらかた理解したが下の計算は何をやってるのか理解できない、、、

        print("norm_source_locations:\n", norm_source_locations[..., None], "\n")
        print("mic_alignments:\n", mic_alignments[:, None, :])

        steering_phase = np.einsum('k,ism,ism->ksm', 2.j*np.pi/sound_speed*freqs, norm_source_locations[..., None], mic_alignments[:, None, :])
        steering_vector = 1./np.sqrt(n_channels)*np.exp(steering_phase)

        return steering_vector

        # steering_phase_2 = np.einsum('k,ism,ims->ksm', 2.j*np.pi/sound_speed*freqs, norm_source_locations[..., None], mic_alignments[:, None, :])
        # steering_vector_2 =1./np.sqrt(n_channels)*np.exp(steering_phase_2)

        # return([steering_vector, steering_phase_2])

#2バイトに変換してファイルに保存
#signal: time-domain 1d array (float)
#file_name: 出力先のファイル名
#sample_rate: サンプリングレート
def write_file_from_time_signal(signal,file_name,sample_rate):
    #2バイトのデータに変換
    signal=signal.astype(np.int16)

    #waveファイルに書き込む
    wave_out = wave.open(file_name, 'w')

    #モノラル:1、ステレオ:2
    wave_out.setnchannels(1)

    #サンプルサイズ2byte
    wave_out.setsampwidth(2)

    #サンプリング周波数
    wave_out.setframerate(sample_rate)

    #データを書き込み
    wave_out.writeframes(signal)

    #ファイルを閉じる
    wave_out.close()


# 遅延和アレイの実験コード
np.random.seed(0)

clean_wave_files=["./CMU_ARCTIC/cmu_us_aew_arctic/wav/arctic_a0001.wav"]

n_sources = len(clean_wave_files)

n_samples = 0


# 最小のフレーム数を取得
for clean_wave_file in clean_wave_files:
    wav = wave.open(clean_wave_file)
    if n_samples < wav.getnframes():
        n_samples = wav.getnframes()
    wav.close()

clean_data = np.zeros([n_sources, n_samples])

s = 0
# ファイルを開いているが、マルチチャンネル（複数マイクで収録した音）はどのように処理されているのか？wavに対する関数の処理を含めて調べる必要がある
for clean_wave_file in clean_wave_files:
    wav = wave.open(clean_wave_file)
    data = wav.readframes(wav.getnframes())
    data = np.frombuffer(data, dtype=np.int16)
    data = data/np.iinfo(np.int16).max
    clean_data[s, :wav.getnframes()] = data
    wav.close()
    s = s + 1

#----------シミュレーションのパラメータ--------------

# サンプリング周波数
sample_rate = 16000

# フレームサイズ
N = 1024

# 周波数の数
Nk = N/2 + 1

# 各ビンの周波数
freqs = np.arange(0, Nk, 1) * sample_rate / N

# 音声と雑音との比率 [dB]
SNR = 20.

# 部屋の大きさ
room_dim = np.r_[10.0, 10.0, 10.0]

# マイクロホンアレイを置く部屋の場所
mic_array_loc = room_dim / 2 + np.random.randn(3) * 0.1

"""
部屋の大きさやマイクロホン位置等は音声データをシミュレーション上で生成するために設定しているものっぽい
つまり、実データではマイクロホン座標を原点として、各方向のステアリングベクトルをスキャンすればOK?
"""

# マイクロホンアレイのマイクロホン配置
mic_alignments = np.array(
    [
        [-0.01, 0.0, 0.0],
        [0.01, 0.0, 0.0],
    ]
)

# マイクロホン数
n_channels = np.shape(mic_alignments)[0]

# get the microphone array
R = mic_alignments .T + mic_array_loc[:, None]

#------------------------------------------------

# 部屋を生成する
room = pa.ShoeBox(room_dim, fs=sample_rate, max_order=0)

# 用いるマイクロホンアレイの情報を設定する
room.add_microphone_array(pa.MicrophoneArray(R, fs=room.fs))

# 音源の場所
doas = np.array(
    [[np.pi/2., 0]]
)

# ↑音源は1つなので1個

# 音源とマイクロホンの距離
distance = 1

source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
source_locations[0, :] = np.cos(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[1, :] = np.sin(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[2, :] = np.cos(doas[:, 0]) 
source_locations *= distance
source_locations += mic_array_loc[:, None]

print(np.shape(clean_data))

# 各音源をシミュレーションに追加する
for s in range(n_sources):
    clean_data[s] /= np.std(clean_data[s])
    room.add_source(source_locations[:, s], signal=clean_data[s])

# シミュレーションを回す
room.simulate(snr=SNR)

# 畳み込んだ波形を取得する（チャンネル、サンプル）
multi_conv_data = room.mic_array.signals

write_file_from_time_signal(multi_conv_data[0] * np.iinfo(np.int16).max/20., "./mix_in.wav", sample_rate)

far_steering_vectors = calculate_steering_vector(R, source_locations=source_locations, freqs=freqs, is_use_far=True)

# 短時間フーリエ変換を行う
f, t, stft_data = sp.stft(multi_conv_data, fs=sample_rate, window="hann", nperseg=N)

# 遅延和アレイを実行する
s_hat = np.einsum("ksm,mkt->skt", np.conjugate(far_steering_vectors), stft_data)

c_hat = np.einsum("skt,ksm->mskt", s_hat, far_steering_vectors)

# 時間領域の波形に戻す
t, ds_out = sp.istft(c_hat[0], fs=sample_rate, window="hann", nperseg=N)

# 大きさを調整する
ds_out = ds_out*np.iinfo(np.int16).max/20.

# ファイルに書き込む
write_file_from_time_signal(ds_out, "./ds_out.wav", sample_rate)




(1, 62081)
norm_source_locations:
 [[[0.65272756]]

 [[0.53263298]]

 [[0.53874747]]] 

mic_alignments:
 [[[5.16640523 5.18640523]]

 [[5.04001572 5.04001572]]

 [[5.0978738  5.0978738 ]]]


In [8]:
# 方向をスキャンするコードを生成
# マイクの位置を0としたシミュレーションの音声波形を生成
# スキャンしたスペクトルを描画->スペクトルはどの値をとる？
#    ->音のアレイ信号処理 p.103より、スペクトルはステアリング方向を変化させたときの各角度についての平均出力パワーをスペクトルとすればいいらしい。
import numpy as np

angles = np.linspace(-np.pi, np.pi, 20)
angles = np.hstack((angles[:, None], np.zeros((angles.size, 1))))
print(angles.shape)

# doas = np.array()

sample_rate = 16000
N = 1024 # フレームサイズ

multi_conv_data = room.mic_array.signals # 時間領域でデータを取得 shapeは(M, 時間インデックス数)

f,t,stft_data = sp.stft(multi_conv_data, fs=sample_rate, window="hann", nperseg=N)

# スペクトル生成のためのフレーム数
frame_num = 10

multi_conv_data_inv = np.transpose(multi_conv_data, axes=(1, 0, 2)).conj()

set_index = 1
while True:
    mean_start_index = (set_index-1)*frame_num
    mean_end_index = set_index*frame_num
    
    if mean_start_index >= multi_conv_data.shape[2]:
        break
    if mean_end_index > multi_conv_data.shape[2]:
        mean_end_index = multi_conv_data.shape[2]
    
    zz = np.einsum("mkt,knt->mn", multi_conv_data[:, :, mean_start_index:mean_end_index], multi_conv_data_inv[:, :, mean_start_index:mean_end_index]) / frame_num
    
    for ("各方向をスキャンする処理")



(20, 2)
